In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib as contrib
from tensorflow.examples.tutorials.mnist import input_data

import argparse
import os
import time

# comment out for less info during the training runs.
tf.logging.set_verbosity(tf.logging.INFO)

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
DATA_DIR = "/tmp/MNIST_data"

# read in data, downloading first as necessary
DATA_SETS = input_data.read_data_sets(DATA_DIR)

# define a utility function for generating a new directory in which to save 
# model information, so multiple training runs don't stomp on each other.
def getNewPath(name=""):
    base="/tmp/tfmodels/mnist_tflearn"
    logpath = os.path.join(base, name + "_" + str(int(time.time())))
    print("Logging to {}".format(logpath))
    return logpath

Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
BATCH_SIZE = 40

def generate_input_fn(dataset, batch_size = BATCH_SIZE):
    def _input_fn():
        X = tf.constant(dataset.images[dataset.labels < 5])
        Y = tf.constant(dataset.labels[dataset.labels < 5].astype(np.int64))
        image_batch, label_batch = tf.train.shuffle_batch([X, Y],
                                                         batch_size = batch_size,
                                                         capacity = 8*batch_size,
                                                         min_after_dequeue=4*batch_size,
                                                         enqueue_many=True)
        return {'pixels': image_batch}, label_batch
    return _input_fn

In [4]:
n_hidden = 100
n_inputs = 28 * 28
n_outputs = 5

reset_graph()

In [ ]:
def elu(z, alpha=1):
    return np.where(z<0, alpha*(np.exp(z)-1), z)

In [ ]:
he_init = contrib.layers.variance_scaling_initializer()

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [ ]:
from functools import partial

my_dense_layer = partial(tf.layers.dense, units=n_hidden, kernel_initializer=he_init, activation=tf.nn.elu)
with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, name="hidden1" )
    hidden2 = my_dense_layer(hidden1, name="hidden2")
    hidden3 = my_dense_layer(hidden2, name="hidden3")
    hidden4 = my_dense_layer(hidden3, name="hidden4")
    hidden5 = my_dense_layer(hidden4, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, kernel_initializer=he_init, name="outputs")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

In [ ]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [ ]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data")
mnist = DATA_SETS

In [ ]:
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

In [ ]:
n_epochs = 10001
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))
        
with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

In [11]:
validation_monitor = contrib.learn.monitors.ValidationMonitor(
    input_fn=generate_input_fn(DATA_SETS.validation, batch_size=20),
    early_stopping_rounds=10,
    eval_steps=1
    )

def define_and_run_dnn_classifier(num_steps, logdir, lr=.01, batch_size=20):
    feature_columns = [contrib.layers.real_valued_column("pixels", dimension=784)]
    
    classifier = tf.contrib.learn.DNNClassifier(
        feature_columns=feature_columns,
        n_classes=5,
        hidden_units=[100,100,100],
        optimizer=tf.train.AdamOptimizer(learning_rate=lr),
        model_dir=logdir
        )
    
    classifier.fit(input_fn=generate_input_fn(DATA_SETS.train, batch_size=batch_size), steps=num_steps,
                  monitors=[validation_monitor])
#     classifier.fit(input_fn=generate_input_fn(DATA_SETS.train, batch_size=batch_size), steps=num_steps)
    
    print("Finished running the deep training via the fit() method")
    
    accuracy_score = classifier.evaluate(input_fn=generate_input_fn(DATA_SETS.test,batch_size=batch_size),
                                         steps=100)['accuracy']
    print('DNN Classifier Accuracy: {0:f}'.format(accuracy_score))

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


In [12]:
print("Running DNN classifier with .01 learning rate...")
classifier = define_and_run_dnn_classifier(200000, getNewPath("deep01"), lr=.01)

Running DNN classifier with .01 learning rate...
Logging to /tmp/tfmodels/mnist_tflearn/deep01_1498108282
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7ae4160c18>, '_task_type': None, '_num_worker_replicas': 0, '_task_id': 0, '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_steps': None, '_environment': 'local', '_keep_checkpoint_max': 5, '_master': '', '_tf_random_seed': None, '_model_dir': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_is_chief': True, '_save_checkpoints_secs': 600, '_save_summary_steps': 100}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to

INFO:tensorflow:global_step/sec: 963.791
INFO:tensorflow:step = 5801, loss = 0.00399236 (0.104 sec)
INFO:tensorflow:global_step/sec: 1006.24
INFO:tensorflow:step = 5901, loss = 0.000759531 (0.099 sec)
INFO:tensorflow:global_step/sec: 1012.87
INFO:tensorflow:step = 6001, loss = 2.33648e-06 (0.099 sec)
INFO:tensorflow:global_step/sec: 1010.19
INFO:tensorflow:step = 6101, loss = 0.165798 (0.099 sec)
INFO:tensorflow:global_step/sec: 996.62
INFO:tensorflow:step = 6201, loss = 0.0139506 (0.100 sec)
INFO:tensorflow:global_step/sec: 988.418
INFO:tensorflow:step = 6301, loss = 0.0779526 (0.101 sec)
INFO:tensorflow:global_step/sec: 996.876
INFO:tensorflow:step = 6401, loss = 0.07982 (0.100 sec)
INFO:tensorflow:global_step/sec: 1002.77
INFO:tensorflow:step = 6501, loss = 0.0781241 (0.100 sec)
INFO:tensorflow:global_step/sec: 986.333
INFO:tensorflow:step = 6601, loss = 0.0355056 (0.101 sec)
INFO:tensorflow:global_step/sec: 941.263
INFO:tensorflow:step = 6701, loss = 0.0271636 (0.106 sec)
INFO:tens

INFO:tensorflow:global_step/sec: 977.962
INFO:tensorflow:step = 14001, loss = 1.38505e-05 (0.102 sec)
INFO:tensorflow:global_step/sec: 980.049
INFO:tensorflow:step = 14101, loss = 0.00239499 (0.102 sec)
INFO:tensorflow:global_step/sec: 976.933
INFO:tensorflow:step = 14201, loss = 0.0021803 (0.102 sec)
INFO:tensorflow:global_step/sec: 991.625
INFO:tensorflow:step = 14301, loss = 0.000924924 (0.101 sec)
INFO:tensorflow:global_step/sec: 971.749
INFO:tensorflow:step = 14401, loss = 0.000199611 (0.103 sec)
INFO:tensorflow:global_step/sec: 967.196
INFO:tensorflow:step = 14501, loss = 0.0849644 (0.103 sec)
INFO:tensorflow:global_step/sec: 988.302
INFO:tensorflow:step = 14601, loss = 0.0092681 (0.101 sec)
INFO:tensorflow:global_step/sec: 969.295
INFO:tensorflow:step = 14701, loss = 0.0302758 (0.103 sec)
INFO:tensorflow:global_step/sec: 983.588
INFO:tensorflow:step = 14801, loss = 0.0572171 (0.102 sec)
INFO:tensorflow:global_step/sec: 983.242
INFO:tensorflow:step = 14901, loss = 0.000329726 (0.

INFO:tensorflow:global_step/sec: 988.526
INFO:tensorflow:step = 22201, loss = 0.0577814 (0.101 sec)
INFO:tensorflow:global_step/sec: 1039.2
INFO:tensorflow:step = 22301, loss = 0.0 (0.096 sec)
INFO:tensorflow:global_step/sec: 989.176
INFO:tensorflow:step = 22401, loss = 2.86102e-07 (0.101 sec)
INFO:tensorflow:global_step/sec: 978.96
INFO:tensorflow:step = 22501, loss = 2.08616e-07 (0.102 sec)
INFO:tensorflow:global_step/sec: 972.752
INFO:tensorflow:step = 22601, loss = 5.28682e-06 (0.103 sec)
INFO:tensorflow:global_step/sec: 1075.06
INFO:tensorflow:step = 22701, loss = 6.59275e-05 (0.093 sec)
INFO:tensorflow:global_step/sec: 1067.84
INFO:tensorflow:step = 22801, loss = 2.98023e-08 (0.094 sec)
INFO:tensorflow:global_step/sec: 1078.65
INFO:tensorflow:step = 22901, loss = 0.00294434 (0.093 sec)
INFO:tensorflow:global_step/sec: 1087.7
INFO:tensorflow:step = 23001, loss = 0.000323136 (0.092 sec)
INFO:tensorflow:global_step/sec: 1090.34
INFO:tensorflow:step = 23101, loss = 0.150147 (0.092 se

INFO:tensorflow:step = 30401, loss = 0.197722 (0.095 sec)
INFO:tensorflow:global_step/sec: 1081.41
INFO:tensorflow:step = 30501, loss = 2.14576e-07 (0.092 sec)
INFO:tensorflow:global_step/sec: 1064.65
INFO:tensorflow:step = 30601, loss = 0.144567 (0.094 sec)
INFO:tensorflow:global_step/sec: 1081.95
INFO:tensorflow:step = 30701, loss = 0.0 (0.092 sec)
INFO:tensorflow:global_step/sec: 1066.35
INFO:tensorflow:step = 30801, loss = 2.08616e-07 (0.094 sec)
INFO:tensorflow:global_step/sec: 1061.48
INFO:tensorflow:step = 30901, loss = 0.0540905 (0.094 sec)
INFO:tensorflow:global_step/sec: 1086.57
INFO:tensorflow:step = 31001, loss = 0.0 (0.092 sec)
INFO:tensorflow:global_step/sec: 1057.8
INFO:tensorflow:step = 31101, loss = 0.00144567 (0.095 sec)
INFO:tensorflow:global_step/sec: 1082.06
INFO:tensorflow:step = 31201, loss = 0.0 (0.092 sec)
INFO:tensorflow:global_step/sec: 1071.95
INFO:tensorflow:step = 31301, loss = 0.00092107 (0.093 sec)
INFO:tensorflow:global_step/sec: 1073.63
INFO:tensorflow

...

INFO:tensorflow:step = 171901, loss = 0.147711 (0.093 sec)
INFO:tensorflow:global_step/sec: 1050.16
INFO:tensorflow:step = 172001, loss = 0.717313 (0.095 sec)
INFO:tensorflow:global_step/sec: 1071.15
INFO:tensorflow:step = 172101, loss = 0.0292867 (0.093 sec)
INFO:tensorflow:global_step/sec: 1069.15
INFO:tensorflow:step = 172201, loss = 0.217673 (0.094 sec)
INFO:tensorflow:global_step/sec: 1080.78
INFO:tensorflow:step = 172301, loss = 0.406565 (0.093 sec)
INFO:tensorflow:global_step/sec: 1059.17
INFO:tensorflow:step = 172401, loss = 0.190056 (0.094 sec)
INFO:tensorflow:global_step/sec: 1081.02
INFO:tensorflow:step = 172501, loss = 0.34037 (0.092 sec)
INFO:tensorflow:global_step/sec: 1050.39
INFO:tensorflow:step = 172601, loss = 0.245747 (0.095 sec)
INFO:tensorflow:global_step/sec: 1059.07
INFO:tensorflow:step = 172701, loss = 0.189347 (0.094 sec)
INFO:tensorflow:global_step/sec: 1053.02
INFO:tensorflow:step = 172801, loss = 0.307045 (0.095 sec)
INFO:tensorflow:global_step/sec: 1057.85


INFO:tensorflow:step = 180101, loss = 0.126503 (0.095 sec)
INFO:tensorflow:global_step/sec: 1068.61
INFO:tensorflow:step = 180201, loss = 0.159309 (0.094 sec)
INFO:tensorflow:global_step/sec: 1082.92
INFO:tensorflow:step = 180301, loss = 0.141492 (0.092 sec)
INFO:tensorflow:global_step/sec: 1031.51
INFO:tensorflow:step = 180401, loss = 0.300314 (0.097 sec)
INFO:tensorflow:global_step/sec: 1058.29
INFO:tensorflow:step = 180501, loss = 0.184124 (0.095 sec)
INFO:tensorflow:global_step/sec: 1073.41
INFO:tensorflow:step = 180601, loss = 0.0192001 (0.093 sec)
INFO:tensorflow:global_step/sec: 1068.94
INFO:tensorflow:step = 180701, loss = 0.257674 (0.094 sec)
INFO:tensorflow:global_step/sec: 1079.09
INFO:tensorflow:step = 180801, loss = 0.07248 (0.093 sec)
INFO:tensorflow:global_step/sec: 1074.37
INFO:tensorflow:step = 180901, loss = 0.220127 (0.093 sec)
INFO:tensorflow:global_step/sec: 1070.44
INFO:tensorflow:step = 181001, loss = 0.124713 (0.093 sec)
INFO:tensorflow:global_step/sec: 1082.33


INFO:tensorflow:step = 188301, loss = 0.15237 (0.093 sec)
INFO:tensorflow:global_step/sec: 1066.57
INFO:tensorflow:step = 188401, loss = 0.0388284 (0.094 sec)
INFO:tensorflow:global_step/sec: 1080.96
INFO:tensorflow:step = 188501, loss = 0.363825 (0.093 sec)
INFO:tensorflow:global_step/sec: 1065.04
INFO:tensorflow:step = 188601, loss = 0.0408202 (0.094 sec)
INFO:tensorflow:global_step/sec: 1080.89
INFO:tensorflow:step = 188701, loss = 0.247624 (0.093 sec)
INFO:tensorflow:global_step/sec: 1059.79
INFO:tensorflow:step = 188801, loss = 0.566252 (0.094 sec)
INFO:tensorflow:global_step/sec: 1073.21
INFO:tensorflow:step = 188901, loss = 0.294562 (0.093 sec)
INFO:tensorflow:global_step/sec: 1079.32
INFO:tensorflow:step = 189001, loss = 0.360696 (0.093 sec)
INFO:tensorflow:global_step/sec: 1079.43
INFO:tensorflow:step = 189101, loss = 0.336409 (0.093 sec)
INFO:tensorflow:global_step/sec: 1069.53
INFO:tensorflow:step = 189201, loss = 0.053528 (0.094 sec)
INFO:tensorflow:global_step/sec: 1052.66

INFO:tensorflow:step = 196501, loss = 0.161705 (0.094 sec)
INFO:tensorflow:global_step/sec: 1080.7
INFO:tensorflow:step = 196601, loss = 0.105956 (0.093 sec)
INFO:tensorflow:global_step/sec: 1081.42
INFO:tensorflow:step = 196701, loss = 0.131406 (0.092 sec)
INFO:tensorflow:global_step/sec: 1086.62
INFO:tensorflow:step = 196801, loss = 0.0168822 (0.092 sec)
INFO:tensorflow:global_step/sec: 1077.39
INFO:tensorflow:step = 196901, loss = 0.220336 (0.093 sec)
INFO:tensorflow:global_step/sec: 1074.96
INFO:tensorflow:step = 197001, loss = 0.179143 (0.093 sec)
INFO:tensorflow:global_step/sec: 1069.16
INFO:tensorflow:step = 197101, loss = 0.0487894 (0.094 sec)
INFO:tensorflow:global_step/sec: 1067.12
INFO:tensorflow:step = 197201, loss = 0.129234 (0.094 sec)
INFO:tensorflow:global_step/sec: 1059.87
INFO:tensorflow:step = 197301, loss = 0.236415 (0.094 sec)
INFO:tensorflow:global_step/sec: 1055.41
INFO:tensorflow:step = 197401, loss = 0.250224 (0.095 sec)
INFO:tensorflow:global_step/sec: 1064.68

DNN Classifier Accuracy: 0.930000


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None

        # extra ops for batch normalization
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [ ]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                fit_params={"X_valid": X_valid1, "y_valid": y_valid1, "n_epochs": 1000},
                                random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1)

In [ ]:
rnd_search.best_params_

In [ ]:
y_pred = rnd_search.predict(X_test1)
accuracy_score(y_test1, y_pred)

In [ ]:
rnd_search.best_estimator_.save("./my_best_mnist_model_0_to_4")

In [ ]:
?RandomizedSearchCV

In [ ]:
??ClassifierMixin